<a href="https://colab.research.google.com/github/Gibbons-Lab/mmmih/blob/main/workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧫🦠 Modeling microbiota-wide metabolism with MICOM

This notebook will accompany the 3rd Microbiome Movement - Maternal and Infant Health workshop. The presentation slides can be [found here](https://gibbons-lab.github.io/mmmih). 

You can save your own local copy of this notebook by using `File > Save a copy in Drive`. You may be promted to cetify the notebook is safe. We promise that it is 🤞

**Disclaimer:**
The linear and quadratic programming problems MICOM has to solve are very large and very complicated. There are some very good commercial solvers that are very expensive (even though they are often free for academic use). To make this tutorial as accessible as possible we will use the Open Source solver [OSQP](https://osqp.org/). OSQP is amazing with quadratic programming problems (kudos!) but not as accurate for linear problems. Solvers usually only guarantee a solution within a certain numerical tolerance of the real solution. In order to make everything work with OSQP this tolerance has to be relaxed to about 10<sup>-3</sup>. This means that any result with an absolute value smaller than that might very well be zero so we should look at larger values only. Installing cost-free academic versions of commercial solvers like [IBM CPLEX](https://www.ibm.com/analytics/cplex-optimizer) or [Gurobi](https://www.gurobi.com/) would alow you to lower the tolerance to 10<sup>-6</sup>.



# 📝 Setup

MICOM installation is is usually pretty straight-forward and can be as easy as typing `pip install micom` into your Terminal.

But first let's start by downloading the materials again and switching to the folder.

In [ ]:
!git clone https://github.com/gibbons-lab/mmmih materials
%cd materials

## Basic Installation

Installing MICOM is straight-forward in Python.

In [ ]:
!pip install -q micom

print("Done! 🎉 ")

Okay, all done. So let's get started 😁.

# 💻 MICOM

We will use the Python interface to MICOM since it plays nicely with Colaboratory. However, you could run the same steps within the QIIME 2 MICOM plugin ([q2-micom](https://library.qiime2.org/plugins/q2-micom/26/)). 

Here is an overview of all the steps and functions:
![micom overview](https://github.com/micom-dev/q2-micom/raw/706f583a060b91c12c0cec7acea2354fdd0dd320/docs/assets/overview.png).

MICOM starts from a combined abundance/taxonomy table, which MICOM abbreviates to a taxonomy table. From that we usually generate a set of community models, one for each sample. 

Because we don't have that much time we will start from a set of already generated models. Those are personalized models for 8 individuals: 4 healthy ones and 4 with recurrent *C. difficile* infections (illustration by Stephanie Swegle). 

![recurrent C. difficile cycle](https://github.com/Gibbons-Lab/mmmih/raw/main/docs/assets/cycle.png)

Those were processed with QIIME 2 and then matched to the AGORA database. The entire build process is described in more details in the [MICOM documentation](https://micom-dev.github.io/micom/high_level.html#Building-and-models-and-simulating-growth).

To get a quick overview over the models we will start by looking at the manifest which is a summary of the build step.


In [ ]:
import pandas as pd

manifest = pd.read_csv("models/manifest.csv")
manifest

We get a summary of the sample IDs, the number of taxa that were found in each sample and how many could be matched to AGORA. We only included taxa that constitute at least 2.5% of the total abundance in each sample here to make everything a bit quicker.





So we now have our community models and can leverage MICOM fully by simulating community growth.

## Simulating growth

With our community models built, we can start to simulate growth with the cooperative tradeoff algorithm. Because we have no diet information for our samples, we will apply the same 'average Western Diet' to each individual. We will start by downloading this diet from the [MICOM data repository](https://zenodo.org/record/3755182/).

In [ ]:
!wget -O western_diet_gut.qza https://zenodo.org/record/3755182/files/western_diet_gut.qza?download=1

This is again a QIIME 2 artifact, which we can load into MICOM.

In [ ]:
from micom.qiime_formats import load_qiime_medium

medium = load_qiime_medium("western_diet_gut.qza")
medium

Many dietary components get absorbed in the small intestine. This medium was created by taking dietary components and depleting all nutrients absorbed in the small intestine by a factor of 10. 

Okay let's go right ahead and simulate growth. This will take a little while and is a great time to ask some questions 🤔

In [ ]:
from micom.workflows import grow
import pickle

growth_results = grow(manifest, "models", medium, tradeoff=0.5, threads=2)

If that takes too long or was aborted, we can read it in from the treasure chest.

In [ ]:
# Will only run if the previous step failed

import pickle

try:
  growth_results
except NameError:
  growth_results = pickle.load(open("treasure_chest/growth.pickle", "rb"))

What kind of results did we get? Well, `grow` returns a tuple of 3 data sets:

1. The predicted growth rate for all taxa in all samples
2. The import and export fluxes for each taxon and the external environment
3. Annotations for the fluxes mapping to other databases

The growth rates are pretty straightforward.

In [ ]:
growth_results.growth_rates.head()

More interesting are the exchange fluxes.

In [ ]:
growth_results.exchanges

So we see how much of each metabolite is either consumed or produced by each taxon in each sample. `tolerance` denotes the accuracy of the solver and tells you the smallest absolute flux that is likely different form zero (i.e. substantial flux). *All of the fluxes are normalized to 1g dry weight of bacteria*. So, you can directly compare fluxes between taxa, even if they are present at very different abundances. 

However, the metabolite names may not be very informative. That's why we have our annotations! For instance, to figure out what `ac[e]` is (air conditioning?), we can do the following:

In [ ]:
anns = growth_results.annotations
anns[anns.metabolite == "ac[e]"]

Ohhh, it's acetate. Yeah that makes more sense 🕵️‍♀️. For the AGORA models you can also use the official VMH knowledge base at https://vmh.life maintained by Dr. Thiele's, lab which will give you rich information on metabolites and reactions. For instance, you can find out a lot more about acetate at: https://www.vmh.life/#metabolite/ac. 

# 📊 Visualizations

Ok, we have seen that we generate a lot of output data from the growth simulations. But how do we make sense of it all? 

We will use the standard visualizations included in MICOM. These tools take in the growth results we obtained before and create visualizations in standalone HTML files that bundle the plots and raw data and can be viewed directly in your browser.

Normally, we could call `viz.view()` afterwards and it would open it in our web browser. However, this will not work in Colab. However, the plot function create the file `growth_rates_[DATE].html` in your `materials` folder. To open it simply download that file and view it in your browser. We can see that there are many things going on, but it's not super clear. Let's continue.

## Growth niches

Two really important questions are 'what dietary nutrients are consumed by the microbiota and what metabolites do the microbiota produce?' We provided nutrients in our medium, but we don't actually know yet what was eaten by the microbiota. Let's check that out using the `plot_exchanges_per_sample` function.

In [ ]:
from micom.viz import *

plot_exchanges_per_sample(growth_results)

You can see that even though all samples have been subjected to the same environmental conditions there are some differences in the set of consumed metabolites. Why do you think that is?

## Metabolic connections to a phenotype

That is all nice, but how does that relate to recurrent *C. diff* infections? To answer that question we can use the `plot_fit` function. This will run a logistic regression with an L1 penalty on coefficients, with our diesase status as the response variable and the normalizeed fluxes as independent variables. In general, import fluxes are not as predictive because, well, they are more relevant to the bacteria than us. What we usually care about (from a host-health perspective) are the production fluxes of metabolites. These are the total production fluxes into the extracellular (lumenal) space, which includes the set of metabolites that are bioavailable to the host. 

We will start by reading the metadata annotations for our samples that tell us which samples belong to which health status.

In [ ]:
metadata = pd.read_table("metadata.tsv").rename(columns={"id": "sample_id"})
metadata.index = metadata.sample_id
metadata


Because OSQP has a somewhat lower solver accuracy, we will be conservative for what we consider to be 'substantial flux' and will filter out fluxes smaller than 0.001 mmol/l.

In [ ]:
from micom.viz import *

pl = plot_fit(growth_results, metadata.disease_stat, atol=1e-3, flux_type="production")

This will again create a file `fit_[DATE].html` that you can open. You will see the production fluxes most predictive of the phenotypes of interest and you can compare them across the groups. In the coefficient plot negative coefficients mean the particular metabolite is produced at a higher flux in the first group (recurrent C. diff in our case), whereas positive coefficients mean it is produced at a higher flux in the second group (healthy). You can plot the fluxes for the individual metabolites in the lower left. 

Which ones do you think can be trusted on which ones can not?

Whatever we conclude here is based on *very* little data so it would be purely hypothetical. However, it may point us to future avenues to explore.